In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

# INTENTO 1 / NUMERICAS CON MEDIA Y CATEGORICAS MODA / GRID Y SMOTE Y RANDOM FOREST

In [2]:
df = pd.read_csv('../df_cleaned.csv')

In [3]:
df.drop('index', axis=1, inplace=True)


In [4]:
df.replace({'<NA>':np.nan}, inplace=True)


In [5]:
df

,Unnamed: 0,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,label
0,0,51,1,1,125.0,213.0,0.0,2.0,125.0,1.0,1.4,1.0,1.0,3.0,0.0
1,1,54,1,3,120.0,237.0,0.0,0.0,150.0,1.0,1.5,NaN,NaN,7.0,2.0
2,2,63,1,4,140.0,0.0,NaN,2.0,149.0,0.0,2.0,1.0,NaN,NaN,2.0
3,3,52,0,2,140.0,NaN,0.0,0.0,140.0,0.0,0.0,NaN,NaN,NaN,0.0
4,4,55,1,4,140.0,217.0,0.0,0.0,111.0,1.0,5.6,3.0,0.0,7.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
911,911,54,1,4,200.0,198.0,0.0,0.0,142.0,1.0,2.0,2.0,NaN,NaN,NaN
912,912,55,1,2,110.0,214.0,1.0,1.0,180.0,0.0,NaN,NaN,NaN,NaN,NaN
913,913,67,1,3,152.0,212.0,0.0,2.0,150.0,0.0,0.8,2.0,0.0,7.0,NaN
914,914,59,1,1,170.0,288.0,0.0,2.0,159.0,0.0,0.2,2.0,0.0,7.0,NaN


In [6]:
df.dtypes


Unnamed: 0      int64
age             int64
sex             int64
cp              int64
trestbps      float64
chol          float64
fbs           float64
restecg       float64
thalach       float64
exang         float64
oldpeak       float64
slope         float64
ca            float64
thal          float64
label         float64
dtype: object

In [7]:
float_cols = ['trestbps', 'chol', 'thalach', 'oldpeak']
categorical_cols = ['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'ca', 'thal']
# Convertir columnas numéricas de nuevo a tipo numérico
df[float_cols] = df[float_cols].astype(float)

# Convertir columnas categóricas de nuevo a tipo categórico
df[categorical_cols] = df[categorical_cols].astype('category')

# Verificar que los dtypes han cambiado
print(df.dtypes)

Unnamed: 0       int64
age              int64
sex           category
cp            category
trestbps       float64
chol           float64
fbs           category
restecg       category
thalach        float64
exang         category
oldpeak        float64
slope         category
ca            category
thal          category
label          float64
dtype: object


In [8]:
df.isnull().sum()


Unnamed: 0      0
age             0
sex             0
cp              0
trestbps       59
chol           30
fbs            89
restecg         2
thalach        55
exang          55
oldpeak        62
slope         308
ca            608
thal          483
label         184
dtype: int64

In [9]:
# Suponiendo que 'df' es tu DataFrame y 'df' es tu DataFrame final
# Convertir '<NA>' a NaN
df.replace('<NA>', np.nan, inplace=True)

# Obtener columnas numéricas y categóricas
numeric_cols = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']
categorical_cols = ['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'ca', 'thal', 'label']

# Imputación para variables numéricas (usando la mediana)
numeric_imputer = SimpleImputer(strategy='median')
df[numeric_cols] = numeric_imputer.fit_transform(df[numeric_cols])

# Imputación para variables categóricas (usando la moda)
categorical_imputer = SimpleImputer(strategy='most_frequent')
df[categorical_cols] = categorical_imputer.fit_transform(df[categorical_cols])

# Verificar que ya no hay valores faltantes
print(df.isnull().sum())

# El DataFrame 'df' ahora contiene los valores imputados


Unnamed: 0    0
age           0
sex           0
cp            0
trestbps      0
chol          0
fbs           0
restecg       0
thalach       0
exang         0
oldpeak       0
slope         0
ca            0
thal          0
label         0
dtype: int64


In [10]:
train_end = df[0:732]
test_end = df[(916-184):]

In [11]:
train_end.isna().sum()

Unnamed: 0    0
age           0
sex           0
cp            0
trestbps      0
chol          0
fbs           0
restecg       0
thalach       0
exang         0
oldpeak       0
slope         0
ca            0
thal          0
label         0
dtype: int64

In [12]:
test_end.isna().sum()

Unnamed: 0    0
age           0
sex           0
cp            0
trestbps      0
chol          0
fbs           0
restecg       0
thalach       0
exang         0
oldpeak       0
slope         0
ca            0
thal          0
label         0
dtype: int64

In [13]:
train_end['label'].value_counts()

0.0    327
1.0    156
2.0    108
3.0    107
4.0     34
Name: label, dtype: int64

In [14]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
import pandas as pd

# Asumiendo que ya tienes cargado tu DataFrame df_train
# Primero, vamos a separar las características (X) de las etiquetas (y)
X = train_end.drop('label', axis=1)  # Eliminar la columna 'label' para obtener las características
y = train_end['label']  # Las etiquetas son los valores de 'label'

# Aplicar SMOTE
smote = SMOTE(random_state=42)
X_res, y_res = smote.fit_resample(X, y)

# Ahora, X_res y y_res son tus características y etiquetas balanceadas, respectivamente


In [15]:
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as imblearnPipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

# Definir explícitamente la tubería
pipeline = imblearnPipeline([
    ('smote', SMOTE(random_state=42)),
    ('classifier', DecisionTreeClassifier())  # Usar 'classifier' como placeholder
])

# Definir el espacio de parámetros para los modelos
param_grid = [
    {
        'classifier': [DecisionTreeClassifier()],
        'classifier__max_depth': [None, 10, 20, 30],
        'classifier__min_samples_leaf': [1, 2, 4]
    },
    {
        'classifier': [RandomForestClassifier()],
        'classifier__n_estimators': [10, 50, 100],
        'classifier__max_depth': [None, 10, 20],
        'classifier__min_samples_leaf': [1, 2, 4]
    }
]

# Configurar GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# Asumiendo que X_res y y_res son tus características y etiquetas balanceadas
grid_search.fit(X_res, y_res)

print("Mejores parámetros:", grid_search.best_params_)
print("Mejor score:", grid_search.best_score_)


Mejores parámetros: {'classifier': RandomForestClassifier(), 'classifier__max_depth': None, 'classifier__min_samples_leaf': 1, 'classifier__n_estimators': 100}
Mejor score: 0.7865443425076453


In [16]:
from sklearn.ensemble import RandomForestClassifier

# Entrenar el modelo con los mejores parámetros encontrados
mejor_modelo = RandomForestClassifier(
    n_estimators=100,
    max_depth=None,
    min_samples_leaf=1,
    random_state=42  # Fijar una semilla para reproducibilidad
)

# Asumiendo que X_res y y_res son tus características y etiquetas balanceadas
mejor_modelo.fit(X_res, y_res)




RandomForestClassifier(random_state=42)

In [17]:
# Excluyendo la columna 'label'
test_end_sin_label = test_end.drop('label', axis=1)

# Ahora puedes hacer predicciones con el conjunto de datos preparado
predicciones = mejor_modelo.predict(test_end_sin_label)

# Y también obtener las probabilidades de cada clase si lo necesitas
probabilidades = mejor_modelo.predict_proba(test_end_sin_label)



In [18]:
# Copiar el DataFrame 'test_end' a 'predicted_df'
predicted_df = test_end.copy()

# Actualizar la columna 'label' con las predicciones
predicted_df['label'] = predicciones

# Mostrar las primeras filas del nuevo DataFrame para verificar
print(predicted_df.head())



     Unnamed: 0   age  sex   cp  trestbps   chol  fbs  restecg  thalach  \
732         732  57.0  1.0  4.0     156.0  173.0  0.0      2.0    119.0   
733         733  52.0  1.0  2.0     160.0  196.0  0.0      0.0    165.0   
734         734  48.0  1.0  2.0     100.0  223.0  0.0      0.0    100.0   
735         735  62.0  1.0  4.0     115.0    0.0  0.0      0.0    128.0   
736         736  51.0  1.0  3.0     110.0  175.0  0.0      0.0    123.0   

     exang  oldpeak  slope   ca  thal  label  
732    1.0      3.0    3.0  0.0   3.0    4.0  
733    0.0      0.0    2.0  0.0   3.0    0.0  
734    0.0      0.0    2.0  0.0   3.0    0.0  
735    1.0      2.5    3.0  0.0   3.0    2.0  
736    0.0      0.6    1.0  0.0   3.0    0.0  


In [19]:
predicted_df

,Unnamed: 0,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,label
732,732,57.0,1.0,4.0,156.0,173.0,0.0,2.0,119.0,1.0,3.0,3.0,0.0,3.0,4.0
733,733,52.0,1.0,2.0,160.0,196.0,0.0,0.0,165.0,0.0,0.0,2.0,0.0,3.0,0.0
734,734,48.0,1.0,2.0,100.0,223.0,0.0,0.0,100.0,0.0,0.0,2.0,0.0,3.0,0.0
735,735,62.0,1.0,4.0,115.0,0.0,0.0,0.0,128.0,1.0,2.5,3.0,0.0,3.0,2.0
736,736,51.0,1.0,3.0,110.0,175.0,0.0,0.0,123.0,0.0,0.6,1.0,0.0,3.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
911,911,54.0,1.0,4.0,200.0,198.0,0.0,0.0,142.0,1.0,2.0,2.0,0.0,3.0,3.0
912,912,55.0,1.0,2.0,110.0,214.0,1.0,1.0,180.0,0.0,0.5,2.0,0.0,3.0,0.0
913,913,67.0,1.0,3.0,152.0,212.0,0.0,2.0,150.0,0.0,0.8,2.0,0.0,7.0,0.0
914,914,59.0,1.0,1.0,170.0,288.0,0.0,2.0,159.0,0.0,0.2,2.0,0.0,7.0,0.0


In [20]:
# Restablecer el índice de predicted_df para que comience en 0 y luego sumar 1 para comenzar desde 1
predicted_df.reset_index(drop=True, inplace=True)
predicted_df['ID'] = predicted_df.index 

# Seleccionar las columnas 'ID' y 'label' para el nuevo DataFrame df_randomf
df_randomf = predicted_df[['ID', 'label']]

# Guardar df_randomf en un archivo CSV sin el índice de pandas
df_randomf.to_csv('try6_data.csv', index=False)


In [21]:
# Excluyendo la columna 'label'
test_end_sin_label = test_end.drop('label', axis=1)

# Ahora puedes hacer predicciones con el conjunto de datos preparado
predicciones = mejor_modelo.predict(test_end_sin_label)

# Y también obtener las probabilidades de cada clase si lo necesitas
probabilidades = mejor_modelo.predict_proba(test_end_sin_label)



In [22]:
# Excluyendo la columna 'label'
test_end_sin_label = test_end.drop('label', axis=1)

# Ahora puedes hacer predicciones con el conjunto de datos preparado
predicciones = mejor_modelo.predict(test_end_sin_label)

# Y también obtener las probabilidades de cada clase si lo necesitas
probabilidades = mejor_modelo.predict_proba(test_end_sin_label)



In [23]:
# Copiar el DataFrame 'test_end' a 'predicted_df'
predicted_df = test_end.copy()

# Actualizar la columna 'label' con las predicciones
predicted_df['label'] = predicciones

# Mostrar las primeras filas del nuevo DataFrame para verificar
print(predicted_df.head())



     Unnamed: 0   age  sex   cp  trestbps   chol  fbs  restecg  thalach  \
732         732  57.0  1.0  4.0     156.0  173.0  0.0      2.0    119.0   
733         733  52.0  1.0  2.0     160.0  196.0  0.0      0.0    165.0   
734         734  48.0  1.0  2.0     100.0  223.0  0.0      0.0    100.0   
735         735  62.0  1.0  4.0     115.0    0.0  0.0      0.0    128.0   
736         736  51.0  1.0  3.0     110.0  175.0  0.0      0.0    123.0   

     exang  oldpeak  slope   ca  thal  label  
732    1.0      3.0    3.0  0.0   3.0    4.0  
733    0.0      0.0    2.0  0.0   3.0    0.0  
734    0.0      0.0    2.0  0.0   3.0    0.0  
735    1.0      2.5    3.0  0.0   3.0    2.0  
736    0.0      0.6    1.0  0.0   3.0    0.0  


In [24]:
predicted_df

,Unnamed: 0,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,label
732,732,57.0,1.0,4.0,156.0,173.0,0.0,2.0,119.0,1.0,3.0,3.0,0.0,3.0,4.0
733,733,52.0,1.0,2.0,160.0,196.0,0.0,0.0,165.0,0.0,0.0,2.0,0.0,3.0,0.0
734,734,48.0,1.0,2.0,100.0,223.0,0.0,0.0,100.0,0.0,0.0,2.0,0.0,3.0,0.0
735,735,62.0,1.0,4.0,115.0,0.0,0.0,0.0,128.0,1.0,2.5,3.0,0.0,3.0,2.0
736,736,51.0,1.0,3.0,110.0,175.0,0.0,0.0,123.0,0.0,0.6,1.0,0.0,3.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
911,911,54.0,1.0,4.0,200.0,198.0,0.0,0.0,142.0,1.0,2.0,2.0,0.0,3.0,3.0
912,912,55.0,1.0,2.0,110.0,214.0,1.0,1.0,180.0,0.0,0.5,2.0,0.0,3.0,0.0
913,913,67.0,1.0,3.0,152.0,212.0,0.0,2.0,150.0,0.0,0.8,2.0,0.0,7.0,0.0
914,914,59.0,1.0,1.0,170.0,288.0,0.0,2.0,159.0,0.0,0.2,2.0,0.0,7.0,0.0


In [25]:
# Restablecer el índice de predicted_df para que comience en 0 y luego sumar 1 para comenzar desde 1
predicted_df.reset_index(drop=True, inplace=True)
predicted_df['ID'] = predicted_df.index 

# Seleccionar las columnas 'ID' y 'label' para el nuevo DataFrame df_randomf
df_randomf = predicted_df[['ID', 'label']]

# Guardar df_randomf en un archivo CSV sin el índice de pandas
df_randomf.to_csv('try6_data.csv', index=False)


In [26]:
# Copiar el DataFrame 'test_end' a 'predicted_df'
predicted_df = test_end.copy()

# Actualizar la columna 'label' con las predicciones
predicted_df['label'] = predicciones

# Mostrar las primeras filas del nuevo DataFrame para verificar
print(predicted_df.head())



     Unnamed: 0   age  sex   cp  trestbps   chol  fbs  restecg  thalach  \
732         732  57.0  1.0  4.0     156.0  173.0  0.0      2.0    119.0   
733         733  52.0  1.0  2.0     160.0  196.0  0.0      0.0    165.0   
734         734  48.0  1.0  2.0     100.0  223.0  0.0      0.0    100.0   
735         735  62.0  1.0  4.0     115.0    0.0  0.0      0.0    128.0   
736         736  51.0  1.0  3.0     110.0  175.0  0.0      0.0    123.0   

     exang  oldpeak  slope   ca  thal  label  
732    1.0      3.0    3.0  0.0   3.0    4.0  
733    0.0      0.0    2.0  0.0   3.0    0.0  
734    0.0      0.0    2.0  0.0   3.0    0.0  
735    1.0      2.5    3.0  0.0   3.0    2.0  
736    0.0      0.6    1.0  0.0   3.0    0.0  


In [27]:
predicted_df

,Unnamed: 0,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,label
732,732,57.0,1.0,4.0,156.0,173.0,0.0,2.0,119.0,1.0,3.0,3.0,0.0,3.0,4.0
733,733,52.0,1.0,2.0,160.0,196.0,0.0,0.0,165.0,0.0,0.0,2.0,0.0,3.0,0.0
734,734,48.0,1.0,2.0,100.0,223.0,0.0,0.0,100.0,0.0,0.0,2.0,0.0,3.0,0.0
735,735,62.0,1.0,4.0,115.0,0.0,0.0,0.0,128.0,1.0,2.5,3.0,0.0,3.0,2.0
736,736,51.0,1.0,3.0,110.0,175.0,0.0,0.0,123.0,0.0,0.6,1.0,0.0,3.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
911,911,54.0,1.0,4.0,200.0,198.0,0.0,0.0,142.0,1.0,2.0,2.0,0.0,3.0,3.0
912,912,55.0,1.0,2.0,110.0,214.0,1.0,1.0,180.0,0.0,0.5,2.0,0.0,3.0,0.0
913,913,67.0,1.0,3.0,152.0,212.0,0.0,2.0,150.0,0.0,0.8,2.0,0.0,7.0,0.0
914,914,59.0,1.0,1.0,170.0,288.0,0.0,2.0,159.0,0.0,0.2,2.0,0.0,7.0,0.0


In [28]:
# Restablecer el índice de predicted_df para que comience en 0 y luego sumar 1 para comenzar desde 1
predicted_df.reset_index(drop=True, inplace=True)
predicted_df['ID'] = predicted_df.index 

# Seleccionar las columnas 'ID' y 'label' para el nuevo DataFrame df_randomf
df_randomf = predicted_df[['ID', 'label']]

# Guardar df_randomf en un archivo CSV sin el índice de pandas
df_randomf.to_csv('try6_data.csv', index=False)


In [29]:
try3_df = pd.read_csv('try6_data.csv')

## INTENTO 2 / ESTANDARIZAR / PARAMETROS CHOL / THAL

In [30]:
df2 = pd.read_csv('../df_cleaned.csv')

In [31]:
df2

,Unnamed: 0,index,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,label
0,0,0,51,1,1,125.0,213.0,0.0,2.0,125.0,1.0,1.4,1.0,1.0,3.0,0.0
1,1,1,54,1,3,120.0,237.0,0.0,0.0,150.0,1.0,1.5,NaN,NaN,7.0,2.0
2,2,2,63,1,4,140.0,0.0,NaN,2.0,149.0,0.0,2.0,1.0,NaN,NaN,2.0
3,3,3,52,0,2,140.0,NaN,0.0,0.0,140.0,0.0,0.0,NaN,NaN,NaN,0.0
4,4,4,55,1,4,140.0,217.0,0.0,0.0,111.0,1.0,5.6,3.0,0.0,7.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
911,911,179,54,1,4,200.0,198.0,0.0,0.0,142.0,1.0,2.0,2.0,NaN,NaN,NaN
912,912,180,55,1,2,110.0,214.0,1.0,1.0,180.0,0.0,NaN,NaN,NaN,NaN,NaN
913,913,181,67,1,3,152.0,212.0,0.0,2.0,150.0,0.0,0.8,2.0,0.0,7.0,NaN
914,914,182,59,1,1,170.0,288.0,0.0,2.0,159.0,0.0,0.2,2.0,0.0,7.0,NaN


In [32]:
df2.drop('index', axis=1, inplace=True)

In [33]:
df2.head()

,Unnamed: 0,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,label
0,0,51,1,1,125.0,213.0,0.0,2.0,125.0,1.0,1.4,1.0,1.0,3.0,0.0
1,1,54,1,3,120.0,237.0,0.0,0.0,150.0,1.0,1.5,NaN,NaN,7.0,2.0
2,2,63,1,4,140.0,0.0,NaN,2.0,149.0,0.0,2.0,1.0,NaN,NaN,2.0
3,3,52,0,2,140.0,NaN,0.0,0.0,140.0,0.0,0.0,NaN,NaN,NaN,0.0
4,4,55,1,4,140.0,217.0,0.0,0.0,111.0,1.0,5.6,3.0,0.0,7.0,3.0


In [34]:
# Suponiendo que 'df' es tu DataFrame y 'df' es tu DataFrame final
# Convertir '<NA>' a NaN
df2.replace('<NA>', np.nan, inplace=True)

# Obtener columnas numéricas y categóricas
numeric_cols2 = ['age', 'trestbps', 'thalach', 'oldpeak']
categorical_cols2 = ['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'ca', 'label']

# Imputación para variables numéricas (usando la mediana)
numeric_imputer2 = SimpleImputer(strategy='mean')
df2[numeric_cols2] = numeric_imputer2.fit_transform(df2[numeric_cols2])

# Imputación para variables categóricas (usando la moda)
categorical_imputer2 = SimpleImputer(strategy='most_frequent')
df2[categorical_cols2] = categorical_imputer2.fit_transform(df2[categorical_cols2])

# Verificar que ya no hay valores faltantes
print(df2.isnull().sum())

# El DataFrame 'df' ahora contiene los valores imputados


Unnamed: 0      0
age             0
sex             0
cp              0
trestbps        0
chol           30
fbs             0
restecg         0
thalach         0
exang           0
oldpeak         0
slope           0
ca              0
thal          483
label           0
dtype: int64


In [35]:
# Función para categorizar los valores de chol
def categorizar_chol(valor):
    if valor == 0 or pd.isnull(valor):  
        return 0
    elif valor < 200:
        return 1
    elif 200 <= valor <= 239:
        return 2
    else:  # valor >= 240
        return 3

# Aplicar la función al DataFrame
df2['chol'] = df2['chol'].apply(categorizar_chol)

# Mostrar los primeros registros para verificar el resultado
print(df2['chol'].head())


0    2
1    2
2    0
3    0
4    2
Name: chol, dtype: int64


In [36]:
def sust_thal(valor):
    if pd.isnull(valor):
        return 0
    else:
        return valor

df2['thal'] = df2['thal'].apply(sust_thal)
print(df2['thal'].head())

0    3.0
1    7.0
2    0.0
3    0.0
4    7.0
Name: thal, dtype: float64


In [37]:
from sklearn.preprocessing import StandardScaler

# Crear un objeto StandardScaler
scaler = StandardScaler()

# Lista de columnas a estandarizar
columns_to_scale2 = ['age', 'trestbps', 'thalach', 'oldpeak']

# Ajustar el escalador solo a las columnas especificadas y transformarlas
df2[columns_to_scale2] = scaler.fit_transform(df2[columns_to_scale2])


In [38]:
train_end2 = df2[0:732]
test_end2 = df2[(916-184):]

In [39]:
train_end2.isna().sum()

Unnamed: 0    0
age           0
sex           0
cp            0
trestbps      0
chol          0
fbs           0
restecg       0
thalach       0
exang         0
oldpeak       0
slope         0
ca            0
thal          0
label         0
dtype: int64

In [40]:
test_end2.isna().sum()

Unnamed: 0    0
age           0
sex           0
cp            0
trestbps      0
chol          0
fbs           0
restecg       0
thalach       0
exang         0
oldpeak       0
slope         0
ca            0
thal          0
label         0
dtype: int64

In [41]:
train_end2['label'].value_counts()

0.0    327
1.0    156
2.0    108
3.0    107
4.0     34
Name: label, dtype: int64

In [42]:
from sklearn.model_selection import train_test_split

X2 = train_end2.drop('label', axis=1)  # Todas las columnas excepto la columna objetivo
Y2 = train_end2['label']

# Dividimos el conjunto de entrenamiento en nuevo entrenamiento y validación (85% entrenamiento, 15% validación del conjunto de entrenamiento original)
X2_train, X2_val, Y2_train, Y2_val = train_test_split(X2, Y2, test_size=0.125, random_state=42)



In [43]:
from imblearn.over_sampling import SMOTE

# Crear un objeto SMOTE
smote2 = SMOTE(random_state=42)

# Aplicar SMOTE solo al conjunto de entrenamiento
X2_train_smote, Y2_train_smote = smote2.fit_resample(X2_train, Y2_train)

# Ahora, X1_train_smote y Y1_train_smote son tus nuevas características y etiquetas de entrenamiento, respectivamente,
# con las clases minoritarias sobremuestreadas.


In [44]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.svm import SVC

# Espacio de parámetros para cada modelo
model_params2 = {
    'KNN': {
        'model': KNeighborsClassifier(),
        'params': {
            'n_neighbors': [3, 5, 7, 9]
        }
    },
    'Random_Forest': {
        'model': RandomForestClassifier(),
        'params': {
            'n_estimators': [10, 50, 100],
            'max_depth': [None, 10, 20, 30]
        }
    },
    'Logistic_Regression': {
        'model': LogisticRegression(),
        'params': {
            'C': [0.1, 1, 10]
        }
    },
    'SVM': {
        'model': SVC(),
        'params': {
            'C': [0.1, 1, 10],
            'kernel': ['rbf', 'linear']
        }
    }
}

# Nota: La regresión lineal no tiene hiperparámetros que suelan ajustarse en el sentido tradicional, por lo que no se incluye en el Grid Search.


In [45]:
from sklearn.ensemble import RandomForestClassifier

# Para demostrar el proceso, ejecutaremos GridSearchCV solo para Random Forest debido a limitaciones del entorno de ejecución
model_to_run2 = 'Random_Forest'
model = model_params2[model_to_run2]['model']
params = model_params2[model_to_run2]['params']

grid_search2 = GridSearchCV(model, params, cv=5, return_train_score=False)
grid_search2.fit(X2_train_smote, Y2_train_smote)

# Mejores parámetros y mejor puntuación - como ejemplo, esto no se ejecutará, solo para ilustración
best_params = grid_search2.best_params_
best_score = grid_search2.best_score_
(best_params, best_score)



({'max_depth': None, 'n_estimators': 100}, 0.7830985915492958)

In [46]:
from sklearn.ensemble import RandomForestClassifier

# Definir el modelo de Random Forest con los parámetros seleccionados
rf_model = RandomForestClassifier(max_depth=None, n_estimators=100, random_state=42)

# Entrenar el modelo con tus datos de entrenamiento
rf_model.fit(X2_train_smote, Y2_train_smote)

# El modelo ahora está entrenado y listo para hacer predicciones o para ser evaluado en tu conjunto de validación o prueba


RandomForestClassifier(random_state=42)

In [47]:
from sklearn.metrics import accuracy_score

# Hacer predicciones en el conjunto de validación
Y2_val_pred = rf_model.predict(X2_val)

# Calcular la precisión
accuracy = accuracy_score(Y2_val, Y2_val_pred)

print(f"La precisión del modelo en el conjunto de validación es: {accuracy}")


La precisión del modelo en el conjunto de validación es: 0.4891304347826087


In [48]:
# Excluyendo la columna 'label'
test_end2_sin_label = test_end2.drop('label', axis=1)

# Ahora puedes hacer predicciones con el conjunto de datos preparado
predicciones2 = rf_model.predict(test_end2_sin_label)

# Y también obtener las probabilidades de cada clase si lo necesitas
probabilidades2 = rf_model.predict_proba(test_end2_sin_label)



In [49]:
# Copiar el DataFrame 'test_end' a 'predicted_df'
predicted_df2 = test_end2.copy()

# Actualizar la columna 'label' con las predicciones
predicted_df2['label'] = predicciones2

# Mostrar las primeras filas del nuevo DataFrame para verificar
print(predicted_df2.head())



     Unnamed: 0       age  sex   cp  trestbps  chol  fbs  restecg   thalach  \
732         732  0.369626  1.0  4.0  1.294872     1  0.0      2.0 -0.737374   
733         733 -0.162728  1.0  2.0  1.511971     1  0.0      0.0  1.092822   
734         734 -0.588612  1.0  2.0 -1.744524     0  0.0      0.0 -1.493324   
735         735  0.901980  1.0  4.0 -0.930400     0  0.0      0.0 -0.379292   
736         736 -0.269199  1.0  3.0 -1.201775     1  0.0      0.0 -0.578226   

     exang   oldpeak  slope   ca  thal  label  
732    1.0  2.019703    3.0  0.0   0.0    3.0  
733    0.0 -0.832821    2.0  0.0   0.0    0.0  
734    0.0 -0.832821    2.0  0.0   0.0    0.0  
735    1.0  1.544282    3.0  0.0   0.0    4.0  
736    0.0 -0.262316    1.0  0.0   3.0    1.0  


In [50]:
predicted_df2

,Unnamed: 0,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,label
732,732,0.369626,1.0,4.0,1.294872,1,0.0,2.0,-0.737374,1.0,2.019703,3.0,0.0,0.0,3.0
733,733,-0.162728,1.0,2.0,1.511971,1,0.0,0.0,1.092822,0.0,-0.832821,2.0,0.0,0.0,0.0
734,734,-0.588612,1.0,2.0,-1.744524,0,0.0,0.0,-1.493324,0.0,-0.832821,2.0,0.0,0.0,0.0
735,735,0.901980,1.0,4.0,-0.930400,0,0.0,0.0,-0.379292,1.0,1.544282,3.0,0.0,0.0,4.0
736,736,-0.269199,1.0,3.0,-1.201775,1,0.0,0.0,-0.578226,0.0,-0.262316,1.0,0.0,3.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
911,911,0.050213,1.0,4.0,3.682968,1,0.0,0.0,0.177724,1.0,1.068861,2.0,0.0,0.0,2.0
912,912,0.156684,1.0,2.0,-1.201775,2,1.0,1.0,1.689624,0.0,0.000000,2.0,0.0,0.0,0.0
913,913,1.434334,1.0,3.0,1.077772,2,0.0,2.0,0.496019,0.0,-0.072148,2.0,0.0,7.0,0.0
914,914,0.582567,1.0,1.0,2.054721,3,0.0,2.0,0.854100,0.0,-0.642653,2.0,0.0,7.0,0.0


In [51]:
predicted_df2['label'].value_counts()

0.0    88
3.0    44
1.0    29
2.0    21
4.0     2
Name: label, dtype: int64

In [52]:
# Restablecer el índice de predicted_df para que comience en 0 y luego sumar 1 para comenzar desde 1
predicted_df2.reset_index(drop=True, inplace=True)
predicted_df2['ID'] = predicted_df2.index 

# Seleccionar las columnas 'ID' y 'label' para el nuevo DataFrame df_randomf
df_randomf2 = predicted_df2[['ID', 'label']]

# Guardar df_randomf en un archivo CSV sin el índice de pandas
df_randomf2.to_csv('try14_data.csv', index=False)


## INTENTO 3

In [53]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

In [54]:
df3 = pd.read_csv('../df_cleaned.csv')

In [55]:
df3

,Unnamed: 0,index,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,label
0,0,0,51,1,1,125.0,213.0,0.0,2.0,125.0,1.0,1.4,1.0,1.0,3.0,0.0
1,1,1,54,1,3,120.0,237.0,0.0,0.0,150.0,1.0,1.5,NaN,NaN,7.0,2.0
2,2,2,63,1,4,140.0,0.0,NaN,2.0,149.0,0.0,2.0,1.0,NaN,NaN,2.0
3,3,3,52,0,2,140.0,NaN,0.0,0.0,140.0,0.0,0.0,NaN,NaN,NaN,0.0
4,4,4,55,1,4,140.0,217.0,0.0,0.0,111.0,1.0,5.6,3.0,0.0,7.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
911,911,179,54,1,4,200.0,198.0,0.0,0.0,142.0,1.0,2.0,2.0,NaN,NaN,NaN
912,912,180,55,1,2,110.0,214.0,1.0,1.0,180.0,0.0,NaN,NaN,NaN,NaN,NaN
913,913,181,67,1,3,152.0,212.0,0.0,2.0,150.0,0.0,0.8,2.0,0.0,7.0,NaN
914,914,182,59,1,1,170.0,288.0,0.0,2.0,159.0,0.0,0.2,2.0,0.0,7.0,NaN


In [56]:
df3.drop('index', axis=1, inplace=True)

In [57]:
df3.head()

,Unnamed: 0,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,label
0,0,51,1,1,125.0,213.0,0.0,2.0,125.0,1.0,1.4,1.0,1.0,3.0,0.0
1,1,54,1,3,120.0,237.0,0.0,0.0,150.0,1.0,1.5,NaN,NaN,7.0,2.0
2,2,63,1,4,140.0,0.0,NaN,2.0,149.0,0.0,2.0,1.0,NaN,NaN,2.0
3,3,52,0,2,140.0,NaN,0.0,0.0,140.0,0.0,0.0,NaN,NaN,NaN,0.0
4,4,55,1,4,140.0,217.0,0.0,0.0,111.0,1.0,5.6,3.0,0.0,7.0,3.0


In [58]:
media_thalach = df3['thalach'].mean()

# Rellenar los NaN con la media calculada
df3['thalach'].fillna(value=media_thalach, inplace=True)

# Mostrar los primeros registros para verificar el resultado
print(df3[['thalach']].head())

   thalach
0    125.0
1    150.0
2    149.0
3    140.0
4    111.0


In [59]:
# Suponiendo que 'df' es tu DataFrame y 'df' es tu DataFrame final
# Convertir '<NA>' a NaN
df3.replace('<NA>', np.nan, inplace=True)

# Obtener columnas numéricas y categóricas
numeric_cols3 = ['age', 'oldpeak']
categorical_cols3 = ['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'ca', 'label']

# Imputación para variables numéricas (usando la mediana)
numeric_imputer3 = SimpleImputer(strategy='mean')
df3[numeric_cols3] = numeric_imputer3.fit_transform(df3[numeric_cols3])

# Imputación para variables categóricas (usando la moda)
categorical_imputer3 = SimpleImputer(strategy='most_frequent')
df3[categorical_cols3] = categorical_imputer3.fit_transform(df3[categorical_cols3])

# Verificar que ya no hay valores faltantes
print(df3.isnull().sum())

# El DataFrame 'df' ahora contiene los valores imputados


Unnamed: 0      0
age             0
sex             0
cp              0
trestbps       59
chol           30
fbs             0
restecg         0
thalach         0
exang           0
oldpeak         0
slope           0
ca              0
thal          483
label           0
dtype: int64


In [60]:
# Función para categorizar los valores de chol
def categorizar_chol(valor):
    if valor == 0 or pd.isnull(valor):  
        return 0
    elif valor < 200:
        return 1
    elif 200 <= valor <= 239:
        return 2
    else:  # valor >= 240
        return 3

# Aplicar la función al DataFrame
df3['chol'] = df3['chol'].apply(categorizar_chol)

# Mostrar los primeros registros para verificar el resultado
print(df3['chol'].head())


0    2
1    2
2    0
3    0
4    2
Name: chol, dtype: int64


In [61]:
def sust_thal(valor):
    if pd.isnull(valor):
        return 0
    else:
        return valor

df3['thal'] = df3['thal'].apply(sust_thal)
print(df3['thal'].head())

0    3.0
1    7.0
2    0.0
3    0.0
4    7.0
Name: thal, dtype: float64


In [62]:
def categorizar_trestbps(valor):
    if pd.isnull(valor):
        return 0
    elif valor < 120:
        return 1
    elif 120 <= valor <= 129:
        return 2
    else:  # valor >= 130
        return 3


df3['trestbps'] = df3['trestbps'].apply(categorizar_trestbps)

# Mostrar los primeros registros para verificar el resultado
print(df3['trestbps'].head())


0    2
1    2
2    3
3    3
4    3
Name: trestbps, dtype: int64


In [63]:
def categorizar_thalach(edad, sexo, thalach):
    if pd.isnull(thalach):
        return 0

    # Categorías basadas en la imagen para hombres y mujeres
    categorias_hombres = [(86, float('inf')), (70, 84), (62, 68), (60, float('inf'))]
    categorias_mujeres = [(96, float('inf')), (78, 94), (72, 76), (70, float('inf'))]
    
    # Seleccionar los límites según el sexo
    if sexo == 1.0:  # Hombres
        limites = categorias_hombres
    else:  # Mujeres
        limites = categorias_mujeres
    
    # Asignar categorías según la edad
    if edad < 30:
        rango = 0
    elif 30 <= edad <= 39:
        rango = 1
    elif 40 <= edad <= 49:
        rango = 2
    else:  # edad >= 50
        rango = 3
    
    # Determinar la categoría basada en el valor de thalach
    for i, (inferior, superior) in enumerate(limites):
        if i == rango:
            if thalach <= superior:
                return i + 1  # Devuelve un número de categoría basado en el índice
    return 5  # Devuelve una categoría para cualquier valor de thalach no clasificado anteriormente

# Aplicar la función actualizada al DataFrame
df3['thalach_cat'] = df3.apply(lambda fila: categorizar_thalach(fila['age'], fila['sex'], fila['thalach']), axis=1)

# Mostrar los primeros registros para verificar el resultado
print(df3[['age', 'sex', 'thalach', 'thalach_cat']].head())


    age  sex  thalach  thalach_cat
0  51.0  1.0    125.0            4
1  54.0  1.0    150.0            4
2  63.0  1.0    149.0            4
3  52.0  0.0    140.0            4
4  55.0  1.0    111.0            4


In [64]:
df3.isnull().sum()

Unnamed: 0     0
age            0
sex            0
cp             0
trestbps       0
chol           0
fbs            0
restecg        0
thalach        0
exang          0
oldpeak        0
slope          0
ca             0
thal           0
label          0
thalach_cat    0
dtype: int64

In [65]:
from sklearn.preprocessing import StandardScaler

# Crear un objeto StandardScaler
scaler3 = StandardScaler()

# Lista de columnas a estandarizar
columns_to_scale3 = ['age','oldpeak']

# Ajustar el escalador solo a las columnas especificadas y transformarlas
df3[columns_to_scale3] = scaler3.fit_transform(df3[columns_to_scale3])


In [66]:
train_end3 = df3[0:732]
test_end3 = df3[(916-184):]

In [67]:
train_end3.isna().sum()

Unnamed: 0     0
age            0
sex            0
cp             0
trestbps       0
chol           0
fbs            0
restecg        0
thalach        0
exang          0
oldpeak        0
slope          0
ca             0
thal           0
label          0
thalach_cat    0
dtype: int64

In [68]:
test_end3.isna().sum()

Unnamed: 0     0
age            0
sex            0
cp             0
trestbps       0
chol           0
fbs            0
restecg        0
thalach        0
exang          0
oldpeak        0
slope          0
ca             0
thal           0
label          0
thalach_cat    0
dtype: int64

In [69]:
train_end3['label'].value_counts()

0.0    327
1.0    156
2.0    108
3.0    107
4.0     34
Name: label, dtype: int64

In [70]:
from sklearn.model_selection import train_test_split

X3 = train_end3.drop('label', axis=1)  # Todas las columnas excepto la columna objetivo
Y3 = train_end3['label']

# Dividimos el conjunto de entrenamiento en nuevo entrenamiento y validación (85% entrenamiento, 15% validación del conjunto de entrenamiento original)
X3_train, X3_val, Y3_train, Y3_val = train_test_split(X3, Y3, test_size=0.125, random_state=42)



In [71]:
from imblearn.over_sampling import SMOTE

# Crear un objeto SMOTE
smote3 = SMOTE(random_state=42)

# Aplicar SMOTE solo al conjunto de entrenamiento
X3_train_smote, Y3_train_smote = smote3.fit_resample(X3_train, Y3_train)

# Ahora, X1_train_smote y Y1_train_smote son tus nuevas características y etiquetas de entrenamiento, respectivamente,
# con las clases minoritarias sobremuestreadas.


In [72]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.svm import SVC

# Espacio de parámetros para cada modelo
model_params3 = {
    'KNN': {
        'model': KNeighborsClassifier(),
        'params': {
            'n_neighbors': [3, 5, 7, 9]
        }
    },
    'Random_Forest': {
        'model': RandomForestClassifier(),
        'params': {
            'n_estimators': [10, 50, 100],
            'max_depth': [None, 10, 20, 30]
        }
    },
    'Logistic_Regression': {
        'model': LogisticRegression(),
        'params': {
            'C': [0.1, 1, 10]
        }
    },
    'SVM': {
        'model': SVC(),
        'params': {
            'C': [0.1, 1, 10],
            'kernel': ['rbf', 'linear']
        }
    }
}

# Nota: La regresión lineal no tiene hiperparámetros que suelan ajustarse en el sentido tradicional, por lo que no se incluye en el Grid Search.


In [73]:
from sklearn.model_selection import train_test_split

# Supongamos que ya has aplicado SMOTE a tus datos de entrenamiento y los tienes en X1_train_smote, Y1_train_smote

# Ejemplo con KNN
grid_search3 = GridSearchCV(model_params3['KNN']['model'], model_params3['KNN']['params'], cv=5, return_train_score=False)
grid_search3.fit(X3_train_smote, Y3_train_smote)

# Resultados
(grid_search3.best_params_ ,grid_search3.best_score_)

({'n_neighbors': 3}, 0.5957746478873239)

In [74]:
from sklearn.ensemble import RandomForestClassifier

# Para demostrar el proceso, ejecutaremos GridSearchCV solo para Random Forest debido a limitaciones del entorno de ejecución
model_to_run4 = 'Random_Forest'
model4 = model_params3[model_to_run4]['model']
params4 = model_params3[model_to_run4]['params']

grid_search4 = GridSearchCV(model4, params4, cv=5, return_train_score=False)
grid_search4.fit(X3_train_smote, Y3_train_smote)

# Mejores parámetros y mejor puntuación - como ejemplo, esto no se ejecutará, solo para ilustración
best_params4 = grid_search4.best_params_
best_score4 = grid_search4.best_score_
(best_params4, best_score4)



({'max_depth': None, 'n_estimators': 100}, 0.7859154929577464)

In [75]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

# Asegúrate de definir model_params correctamente como se mostró anteriormente

# Corregimos aquí para ejecutar GridSearchCV para la Regresión Logística
model_to_run5 = 'Logistic_Regression'
model5 = model_params3[model_to_run5]['model']  # Usamos model_to_run2
params5 = model_params3[model_to_run5]['params']

# Crear el objeto GridSearchCV para Regresión Logística
grid_search5 = GridSearchCV(model5, params5, cv=5, return_train_score=False)

# Reemplaza X1_train_smote, Y1_train_smote con tus datos reales de entrenamiento
grid_search5.fit(X3_train_smote, Y3_train_smote)

# Obtener los mejores parámetros y la mejor puntuación
best_params5 = grid_search5.best_params_
best_score5 = grid_search5.best_score_

(best_params5, best_score5)


c:\Users\Pacotm\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\Pacotm\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/

({'C': 1}, 0.4598591549295775)

In [76]:
from sklearn.ensemble import RandomForestClassifier

# Definir el modelo de Random Forest con los parámetros seleccionados
rf_model3 = RandomForestClassifier(max_depth=30, n_estimators=100, random_state=42)

# Entrenar el modelo con tus datos de entrenamiento
rf_model3.fit(X3_train_smote, Y3_train_smote)

# El modelo ahora está entrenado y listo para hacer predicciones o para ser evaluado en tu conjunto de validación o prueba


RandomForestClassifier(max_depth=30, random_state=42)

In [77]:
from sklearn.metrics import accuracy_score

# Hacer predicciones en el conjunto de validación
Y3_val_pred = rf_model3.predict(X3_val)

# Calcular la precisión
accuracy = accuracy_score(Y3_val, Y3_val_pred)

print(f"La precisión del modelo en el conjunto de validación es: {accuracy}")


La precisión del modelo en el conjunto de validación es: 0.532608695652174


In [78]:
# Excluyendo la columna 'label'
test_end3_sin_label = test_end3.drop('label', axis=1)

# Ahora puedes hacer predicciones con el conjunto de datos preparado
predicciones3 = rf_model3.predict(test_end3_sin_label)

# Y también obtener las probabilidades de cada clase si lo necesitas
probabilidades3 = rf_model3.predict_proba(test_end3_sin_label)



In [79]:
# Copiar el DataFrame 'test_end' a 'predicted_df'
predicted_df3 = test_end3.copy()

# Actualizar la columna 'label' con las predicciones
predicted_df3['label'] = predicciones3

# Mostrar las primeras filas del nuevo DataFrame para verificar
print(predicted_df3.head())



     Unnamed: 0       age  sex   cp  trestbps  chol  fbs  restecg  thalach  \
732         732  0.369626  1.0  4.0         3     1  0.0      2.0    119.0   
733         733 -0.162728  1.0  2.0         3     1  0.0      0.0    165.0   
734         734 -0.588612  1.0  2.0         1     0  0.0      0.0    100.0   
735         735  0.901980  1.0  4.0         1     0  0.0      0.0    128.0   
736         736 -0.269199  1.0  3.0         1     1  0.0      0.0    123.0   

     exang   oldpeak  slope   ca  thal  label  thalach_cat  
732    1.0  2.019703    3.0  0.0   0.0    3.0            4  
733    0.0 -0.832821    2.0  0.0   0.0    0.0            4  
734    0.0 -0.832821    2.0  0.0   0.0    0.0            5  
735    1.0  1.544282    3.0  0.0   0.0    2.0            4  
736    0.0 -0.262316    1.0  0.0   3.0    1.0            4  


In [80]:
predicted_df3

,Unnamed: 0,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,label,thalach_cat
732,732,0.369626,1.0,4.0,3,1,0.0,2.0,119.000000,1.0,2.019703,3.0,0.0,0.0,3.0,4
733,733,-0.162728,1.0,2.0,3,1,0.0,0.0,165.000000,0.0,-0.832821,2.0,0.0,0.0,0.0,4
734,734,-0.588612,1.0,2.0,1,0,0.0,0.0,100.000000,0.0,-0.832821,2.0,0.0,0.0,0.0,5
735,735,0.901980,1.0,4.0,1,0,0.0,0.0,128.000000,1.0,1.544282,3.0,0.0,0.0,2.0,4
736,736,-0.269199,1.0,3.0,1,1,0.0,0.0,123.000000,0.0,-0.262316,1.0,0.0,3.0,1.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
911,911,0.050213,1.0,4.0,3,1,0.0,0.0,142.000000,1.0,1.068861,2.0,0.0,0.0,3.0,4
912,912,0.156684,1.0,2.0,1,2,1.0,1.0,180.000000,0.0,0.000000,2.0,0.0,0.0,0.0,4
913,913,1.434334,1.0,3.0,3,2,0.0,2.0,150.000000,0.0,-0.072148,2.0,0.0,7.0,0.0,4
914,914,0.582567,1.0,1.0,3,3,0.0,2.0,159.000000,0.0,-0.642653,2.0,0.0,7.0,0.0,4


In [81]:
predicted_df3['label'].value_counts()

0.0    89
3.0    48
2.0    24
1.0    22
4.0     1
Name: label, dtype: int64

In [82]:
# Restablecer el índice de predicted_df para que comience en 0 y luego sumar 1 para comenzar desde 1
predicted_df3.reset_index(drop=True, inplace=True)
predicted_df3['ID'] = predicted_df3.index 

# Seleccionar las columnas 'ID' y 'label' para el nuevo DataFrame df_randomf
df_randomf3 = predicted_df3[['ID', 'label']]

# Guardar df_randomf en un archivo CSV sin el índice de pandas
df_randomf3.to_csv('try15_data.csv', index=False)
